In [1]:
#Daily Feature
import pandas as pd
import numpy as np
import datetime as datetime
import pytse_client as tse
from pytse_client import download_client_types_records
from pytse_client.download import download_financial_indexes

## Importing price + individuals + total index data from library

In [2]:
#Price data
tickers = tse.download(symbols="فولاد", adjust=True, write_to_csv=True)
df = pd.DataFrame.from_dict(tickers["فولاد"])

In [3]:
#Individual data
if __name__ == '__main__':
    records_dict = download_client_types_records("فولاد")
df1 = pd.DataFrame.from_dict(records_dict["فولاد"])

df1['date'] = pd.to_datetime(df1['date'])
df1.iloc[:, 1: 13] =df1.iloc[:, 1: 13].astype('float')

df_feature = pd.merge(df, df1, how = 'left', on = 'date')

In [ ]:
df_feature.to_csv("base.csv")

In [4]:
#Total index
marketIndex = download_financial_indexes(symbols = 'شاخص كل', write_to_csv = True, include_jdate = True)
marketIndex = marketIndex['شاخص كل']
marketIndex.rename(columns = {'value': 'TEDPIX'}, inplace = True)

df_feature = pd.merge(df_feature, marketIndex, how = 'left', on = 'date')

## Alireza features

In [5]:
#Alireza columns
df_feature['pct_change'] = df_feature['adjClose'].pct_change() * 100
df_feature['Enter_individual_money'] =df_feature['individual_buy_value'] -df_feature['individual_sell_value']
df_feature['Individual_buy_per_capita'] =df_feature['individual_buy_value'] /df_feature['individual_buy_count']
df_feature['Individual_sell_per_capita'] =df_feature['individual_sell_value'] /df_feature['individual_sell_count']
df_feature['Individual_buy_power_to_Individual_sell'] =df_feature['Individual_buy_per_capita'] /df_feature['Individual_sell_per_capita']

## Import daily features

In [6]:
import daily_features as fe

x = fe.candlestick_feature(df_feature)
y = fe.weight_feature(df_feature)
k = fe.proportion_feature(df_feature)
w = fe.logarithmic_feature(df_feature)
z = fe.shift_data(df_feature)

df_feature = pd.concat([df_feature, x, y, k, w, z], axis=1)

## Import indicator features

In [7]:
import indicator_features as fe

a = fe.BB(df_feature)
b = fe.EMA(df_feature)
c = fe.MACD(df_feature)
d = fe.RSI(df_feature)
e = fe.SMA(df_feature)
f = fe.STOCHASTIC(df_feature)

df_feature = pd.concat([df_feature, a, b, c, d, e, f], axis=1)


In [12]:
df_feature.to_csv("file1.csv")

In [11]:
labeling_df = pd.read_feather('file1.feather')
labeling_df.head()

ArrowInvalid: File is too small to be a well-formed file